In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.2.16:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.2.16.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.2.16
Uninstalling opencv-contrib-python-3.4.2.16:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-3.4.2.16.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-3.4.2.16
  Using cached https://files.pythonhosted.org/packages/fa/7d/5042b668a8ed41d2a80b8c172f5efcd572e3c046c75ae029407e19b7fc68/opencv_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/08/f1/66330f4042c4fb3b2d77a159db8e8916d9cdecc29bc8c1f56bc7f8a9bec9/opencv_contrib_python-3.4.2.16-cp36-cp36m-manylinux1_x86_64.whl


In [0]:
pip install kmc2

In [0]:
import os
import cv2
import kmc2
import numpy as np

from glob import glob
from PIL import Image
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from scipy.cluster.vq import vq
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [0]:
!wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
!tar -zxvf 101_ObjectCategories.tar.gz

--2019-11-19 06:07:16--  http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131740031 (126M) [application/x-tar]
Saving to: ‘101_ObjectCategories.tar.gz.3’

101_ObjectCategorie 100%[===================>] 125.64M  12.0MB/s    in 12s     

2019-11-19 06:07:28 (10.7 MB/s) - ‘101_ObjectCategories.tar.gz.3’ saved [131740031/131740031]

101_ObjectCategories/
101_ObjectCategories/BACKGROUND_Google/
101_ObjectCategories/BACKGROUND_Google/tmp
101_ObjectCategories/BACKGROUND_Google/image_0004.jpg
101_ObjectCategories/BACKGROUND_Google/image_0005.jpg
101_ObjectCategories/BACKGROUND_Google/image_0006.jpg
101_ObjectCategories/BACKGROUND_Google/image_0007.jpg
101_ObjectCategories/BACKGROUND_Google/image_0008.jpg
101_ObjectCategories/BACKGROUND_Googl

In [0]:
# sift 선언
sift = cv2.xfeatures2d.SIFT_create()

In [0]:
DATA_ROOT = "./101_ObjectCategories"
label = 0
train_des = list()
train_labels = list()
test_des = list()
test_labels = list()


for cls in tqdm(os.listdir(DATA_ROOT)):
  img_list = os.listdir(DATA_ROOT+'/'+cls)
  img_list.sort()

  for img in img_list[:30]:
    image = cv2.imread(DATA_ROOT+'/'+cls+'/'+img)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)   
    
    step_size = 10
    kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, gray.shape[0], step_size) for x in range(0, gray.shape[1], step_size)]
    keypoint, des = sift.compute(gray, kp)
    
    #kp, des = sift.detectAndCompute(gray,None)
    train_des.append(des)
    train_labels.append(label)

  limit_cnt = 0

  for img in img_list[30:]:
    if limit_cnt == 50 :
      break
    image = cv2.imread(DATA_ROOT+'/'+cls+'/'+img)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    step_size = 10
    kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, gray.shape[0], step_size) for x in range(0, gray.shape[1], step_size)]
    keypoint, des = sift.compute(gray, kp)

    #kp, des = sift.detectAndCompute(gray,None)
    test_des.append(des)
    test_labels.append(label)
    limit_cnt = limit_cnt +1

  label = label + 1

100%|██████████| 102/102 [06:43<00:00,  3.66s/it]


In [0]:
train_des_arr = np.array(train_des[0])
for remaining in tqdm(train_des[1:]):
  train_des_arr = np.vstack((train_des_arr, remaining))

100%|██████████| 3059/3059 [06:36<00:00,  4.01it/s]


In [0]:
seeding = kmc2.kmc2(train_des_arr, 200)
Kmeas = MiniBatchKMeans(200, init=seeding).fit(train_des_arr)
scaler = StandardScaler()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  


In [0]:
histogram_feature_trian = list()
histogram_feature_test = list()

for i in range(len(train_des)) :
  train_histogram = vq(train_des[i],Kmeas.cluster_centers_)
  train_feature = np.histogram(train_histogram[0],bins=list(range(201)))
  histogram_feature_trian.append(train_feature[0])

for i in range(len(test_des)) :
  test_histogram = vq(test_des[i],Kmeas.cluster_centers_)
  test_feature = np.histogram(test_histogram[0],bins=list(range(201)))
  histogram_feature_test.append(test_feature[0])

histogram_feature_trian = np.array(histogram_feature_trian)
histogram_feature_test = np.array(histogram_feature_test)

scaler.fit(histogram_feature_trian)

histogram_feature_trian = scaler.transform(histogram_feature_trian)
histogram_feature_test = scaler.transform(histogram_feature_test)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [0]:
svm_model = svm.SVC(gamma='scale')
svm_model.fit(histogram_feature_trian, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_predict = svm_model.predict(histogram_feature_test)

print(classification_report(test_labels, y_predict))

              precision    recall  f1-score   support

           0       0.47      0.39      0.43        23
           1       0.24      0.10      0.14        50
           2       0.23      0.42      0.30        19
           3       0.59      0.60      0.59        50
           4       0.25      0.27      0.26        15
           5       0.00      0.00      0.00         1
           6       0.47      1.00      0.64         9
           7       0.87      0.90      0.88        50
           8       0.26      0.75      0.39        12
           9       0.44      0.75      0.56        20
          10       0.33      0.41      0.37        34
          11       0.89      0.94      0.91        17
          12       0.12      0.38      0.18         8
          13       0.36      0.16      0.22        50
          14       0.11      0.15      0.12        13
          15       0.03      0.14      0.05         7
          16       0.32      0.48      0.39        25
          17       0.89    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
confusion_matrix(test_labels, y_predict)

array([[ 9,  0,  2, ...,  0,  0,  0],
       [ 1,  5,  0, ...,  3,  0,  1],
       [ 0,  1,  8, ...,  1,  0,  0],
       ...,
       [ 0,  0,  0, ...,  7,  0,  0],
       [ 0,  0,  0, ...,  0, 21,  0],
       [ 0,  0,  0, ...,  1,  0, 16]])